### calculate a rolling 30 days climatology (including terciles and quintiles) from GPM / IMERG over the period 2001 - 2018

In [1]:
%matplotlib inline
from matplotlib import pyplot as plt

In [2]:
import seaborn as sns

In [3]:
sns.set_context('paper')

In [4]:
import os
import sys
from glob import glob
import json
import pathlib

In [5]:
from datetime import datetime, timedelta

In [6]:
from dateutil import parser

In [7]:
import numpy as np
import pandas as pd 

### load the dictionnary 

In [8]:
with open('../../data/dict_countries.json', 'r') as fj: 
    dict_countries = json.load(fj)

### functions_definitions

In [10]:
def plot_climo(climo, window=30, dirname='../figures/climatologies/', country_fname=''): 
    import seaborn as sns
    sns.set_context('talk')
    
    climo = pd.DataFrame(climo.to_numpy(), index=range(1, 366), columns=climo.columns)
    
    f, ax = plt.subplots(figsize=(10,10))
    ax.fill_between(climo.index, climo.loc[:,'33.3%'].values, climo.loc[:,'66.6%'].values, alpha=0.5, label='interquartile range')
    ax.plot(climo.loc[:,'mean'], lw=3, label='mean')
    ax.plot(climo.loc[:,'50%'], lw=3, label='median')
    ax.legend(frameon=False)
    ax.grid(ls=':')
    ax.set_xlim(0, 366)
    ax.set_title("{} days running total for {}".format(window, country_name))
    ax.set_xlabel('day of year (365 days)')
    ax.set_ylabel('GPM / IMERG rainfall (mm/day)')
    f.savefig(os.path.join(dirname, 'climo_GPM_IMERG_{}_days_{}.png'.format(window, country_fname)))
    plt.close(f)

In [11]:
lfiles = glob('../outputs/DF_ROI_daily_*.csv')

In [12]:
lfiles

['../outputs/DF_ROI_daily_GPM_IMERG_rainfall_2001_2018_Samoa.csv',
 '../outputs/DF_ROI_daily_GPM_IMERG_rainfall_2001_2018_Guam.csv',
 '../outputs/DF_ROI_daily_GPM_IMERG_rainfall_2001_2018_Nauru.csv',
 '../outputs/DF_ROI_daily_GPM_IMERG_rainfall_2001_2018_Marquesas.csv',
 '../outputs/DF_ROI_daily_GPM_IMERG_rainfall_2001_2018_Tuvalu.csv',
 '../outputs/DF_ROI_daily_GPM_IMERG_rainfall_2001_2018_Palau.csv',
 '../outputs/DF_ROI_daily_GPM_IMERG_rainfall_2001_2018_Northern_Marianas.csv',
 '../outputs/DF_ROI_daily_GPM_IMERG_rainfall_2001_2018_Marshall_Islands.csv',
 '../outputs/DF_ROI_daily_GPM_IMERG_rainfall_2001_2018_Wallis_and_Futuna.csv',
 '../outputs/DF_ROI_daily_GPM_IMERG_rainfall_2001_2018_Kiribati_Phoenix_Islands.csv',
 '../outputs/DF_ROI_daily_GPM_IMERG_rainfall_2001_2018_Solomon_Islands.csv',
 '../outputs/DF_ROI_daily_GPM_IMERG_rainfall_2001_2018_Vanuatu_South.csv',
 '../outputs/DF_ROI_daily_GPM_IMERG_rainfall_2001_2018_New_Caledonia.csv',
 '../outputs/DF_ROI_daily_GPM_IMERG_rainfall_

In [13]:
len(lfiles)

29

In [15]:
opath = '../outputs/climatologies/'

In [25]:
for filename in lfiles: 
    country_fname = os.path.basename(filename)[42:].split('.')[-2] 
    country_name = dict_countries[country_fname] 
    roi_df = pd.read_csv(filename, index_col=0, parse_dates=True) 
    
    """
    removes the 29th of Feb 
    """
    
    roi_df.loc[(roi_df.index.month == 2) & (roi_df.index.day == 29),:] = np.nan

    roi_df = roi_df.dropna() 
    
    for window in [30,60,90,180]: 
    
        roi_df_rm = roi_df.rolling(window=window, min_periods=window).sum()

        climo = roi_df_rm.groupby([roi_df_rm.index.month, roi_df_rm.index.day]).describe(percentiles=[0.1,0.2,0.3,0.333,0.4,0.5,0.6,0.666,0.7,0.8,0.9])

        climo.index.names = ['month','day']

        climo.columns = climo.columns.droplevel(0) 

        plot_climo(climo, window=window, country_fname=country_fname)

        climo.to_csv(os.path.join(opath, 'GPM_IMERG_climo_{}_days_{}.csv'.format(window, country_fname)))

### calculates the monthly climatologies 

In [26]:
opath = pathlib.Path(opath)

In [27]:
opath

PosixPath('../outputs/climatologies')

In [29]:
for filename in lfiles: 
    country_fname = os.path.basename(filename)[42:].split('.')[-2] 
    country_name = dict_countries[country_fname] 
    roi_df = pd.read_csv(filename, index_col=0, parse_dates=True) 
    roi_df_m = roi_df.resample('1M').mean()
    climo = roi_df_m.groupby([roi_df_m.index.month]).describe(percentiles=[0.1,0.2,0.3,0.333,0.4,0.5,0.6,0.666,0.7,0.8,0.9])
    climo.columns = climo.columns.droplevel(0) 
    climo.index.name = 'month'
    climo.to_csv(opath /  'GPM_IMERG_monthly_climo_{}.csv'.format(country_fname))